In [2]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Import and preprocess

In [5]:
pcd = o3d.io.read_point_cloud("data/_frames_1_to_8.ply")
print(pcd)
o3d.visualization.draw_geometries([pcd])

PointCloud with 180985 points.


# Preprocess

In [3]:
pcd.translate(-pcd.get_center())


PointCloud with 180985 points.

In [4]:
front = [ 0.84407312852327709, -0.47343977579888252, 0.25178429735873387 ]
lookat = [ 1.7430326603208821, -2.1239703901019853, 1.7369356463018559 ]
up = [ -0.25536249399327232, 0.05798759846744491, 0.96510488294563845 ]
zoom = 0.5

In [5]:
filtered_pcd = pcd.remove_statistical_outlier(16, 10)
outliers = pcd.select_by_index(filtered_pcd[1], invert=True)
outliers.paint_uniform_color([1,0,0])
filtered_pcd = filtered_pcd[0]
filtered_pcd.paint_uniform_color([0.6,0.6,0.6])

o3d.visualization.draw_geometries([filtered_pcd, outliers], front=front, lookat=lookat, up=up, zoom=zoom)

In [6]:
nn_distance = np.mean(filtered_pcd.compute_nearest_neighbor_distance())

normal_radius = nn_distance*4

filtered_pcd.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=normal_radius, max_nn=16))
o3d.visualization.draw_geometries([filtered_pcd, outliers], front=front, lookat=lookat, up=up, zoom=zoom, point_show_normal=True)


# Ransac

In [7]:
pt_plane_distance = 0.02

plane_model, inliers = pcd.segment_plane(distance_threshold=pt_plane_distance, ransac_n=3, num_iterations=1000)

inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)
inlier_cloud.paint_uniform_color([0.6,0.6,0.6])

o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud], front=front, lookat=lookat, up=up, zoom=zoom)


# Downsample

In [8]:
downpcd = outlier_cloud.voxel_down_sample(voxel_size=0.1)
#o3d.visualization.draw_geometries([pcd])
print(downpcd)

PointCloud with 1920 points.


# Cluster

In [9]:
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(downpcd.cluster_dbscan(eps=0.5, min_points=10, print_progress=True))

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 2                       ] 2%
Precompute neighbors.[========================================] 100%
Clustering[========================================] 100%         ] 60%


In [10]:
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
downpcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

point cloud has 2 clusters


In [11]:
o3d.visualization.draw_geometries([downpcd, inlier_cloud], front=front, lookat=lookat, up=up, zoom=zoom)

# OpenCV

In [12]:
import cv2 as cv

In [18]:
mesh = o3d.io.read_triangle_mesh("./saves/monkey.ply")
mesh.compute_vertex_normals()
mesh_pcd = mesh.sample_points_uniformly(number_of_points=2000)

o3d.visualization.draw_geometries([mesh, mesh_pcd])

In [19]:
mesh_pcd

PointCloud with 2000 points.

In [34]:
arr = np.asarray(downpcd.points)
print(arr)

[[-3.50453364  2.39421924  1.46416823]
 [-3.2303437   2.38656028  1.11143724]
 [-3.65997779  1.8553532   1.67470471]
 ...
 [-0.01661275  0.327039    1.87270949]
 [ 0.72361356  0.48176678  0.90810801]
 [-2.37852839  1.98468249  1.11617644]]


In [56]:
cluster0 = downpcd.select_by_index(np.where(labels == 0)[0])
cluster1 = downpcd.select_by_index(np.where(labels == 1)[0])

In [58]:
o3d.visualization.draw_geometries([cluster0])